# * Sales Daily

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

In [2]:
''' Input parameter '''

op_dir = 'data'
op_sales_file = f'sales_daily_{str_curr_dt}'

# v_year = 0
# v_month_start = 0
# v_month_end = 0
# v_date = 20250101

print(f'\nParameter input...\n')
print(f'   -> op_dir: {op_dir}')
print(f'   -> op_sales_file: {op_sales_file}')
# print(f'\n   -> v_year: {v_year}')
# print(f'   -> v_month_start: {v_month_start}')
# print(f'   -> v_month_end: {v_month_end}')
# print(f'\n   -> v_date: {v_date}')


Parameter input...

   -> op_dir: data
   -> op_sales_file: sales_daily_20260202


## Import Transaction
-> DIM_CORP_KPI

In [3]:
''' Execute transaction '''


# Input parameter
v_start_date = 20250101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()

query = (f"""
    SELECT /*+PARALLEL(8)*/ 
        TM.TM_KEY_YR, TM.TM_KEY_QTR, TM.TM_KEY_MTH, TM.TM_KEY_WK, A.TM_KEY_DAY
        , A.PRODUCT, A.OWNER_ID, A.METRIC_CD, A.METRIC_NAME, A.COMP_CD, A.VERSION, A.AREA_CD, A.AREA_DESC, A.AREA_TYPE, A.LOAD_DATE, A.REMARK, A.UPD_TM, A.PPN_TM
        , A.METRIC_VALUE AS ACTUAL
        , (TG.METRIC_VALUE / TM.DAYS_IN_MONTH) AS TARGET
        , TG.METRIC_VALUE AS TARGET_MTH
        , TM.DAYS_IN_MONTH, TM.PERIODFLAG
        , O.AREA_NAME, O.AREA_CD_G, O.AREA_NAME_G
        
    FROM CDSAPPO.DIM_CORP_KPI A

    LEFT JOIN GEOSPCAPPO.DIM_TIME_CITRINE TM
        ON TM.TM_KEY_DAY = A.TM_KEY_DAY
        
    LEFT JOIN CDSAPPO.DIM_CORP_TARGET TG
        ON TG.METRIC_CD = A.METRIC_CD
        AND TG.AREA_CD = A.AREA_CD
        AND TG.TM_KEY_DAY = TM.TM_KEY_MTH
        
    LEFT JOIN GEOSPCAPPO.DIM_MOOC_AREA_CITRINE H
        ON H.AREA_TYPE = 'H'
        AND H.AREA_CD = A.AREA_CD
        
    LEFT JOIN (
        SELECT H.AREA_CD, H.AREA_NAME
            , CASE WHEN H.AREA_TYPE = 'H' THEN G.AREA_CD END AREA_CD_G
            , CASE WHEN H.AREA_TYPE = 'H' THEN G.AREA_NAME END AREA_NAME_G
        FROM GEOSPCAPPO.DIM_MOOC_AREA_CITRINE H
        LEFT JOIN (
            SELECT AREA_CD, AREA_NAME
            FROM GEOSPCAPPO.DIM_MOOC_AREA_CITRINE
            WHERE AREA_TYPE = 'G'
        ) G
            ON G.AREA_CD = H.AREA_CD_G
        WHERE AREA_TYPE IN ('P', 'G', 'H')
    ) O
        ON O.AREA_CD = A.AREA_CD
        
    WHERE A.TM_KEY_DAY >= {v_start_date}
    AND REGEXP_LIKE(A.METRIC_CD, 'CS$|CG$') -->> 2025 KPIs
    AND A.AREA_TYPE IN ('P', 'G', 'H')
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            --UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            --UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            --UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            --UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = A.AREA_CD )
""")


try:
    # Create Dataframe
    src_cur.execute(query)
    rows = src_cur.fetchall()
    src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\nDataFrame: {src_df.shape[0]} rows, {src_df.shape[1]} columns')

    # Generate CSV file
    src_df.to_csv(f'{op_dir}/{op_sales_file}.csv', index=False, encoding='utf-8')
    print(f'\n   -> Generate "{op_sales_file}.csv" successfully')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20250101

Data as of 2026-02-02, 11:18:32

DataFrame: 1948894 rows, 26 columns

   -> Generate "sales_daily_20260202.csv" successfully


## Review

In [4]:
src_df

,TM_KEY_YR,TM_KEY_QTR,TM_KEY_MTH,TM_KEY_WK,TM_KEY_DAY,PRODUCT,OWNER_ID,METRIC_CD,METRIC_NAME,COMP_CD,...,UPD_TM,PPN_TM,ACTUAL,TARGET,TARGET_MTH,DAYS_IN_MONTH,PERIODFLAG,AREA_NAME,AREA_CD_G,AREA_NAME_G
0,2025,20254,202511,2025045,20251107,POSTPAID,01035652,DB2R010500CG,Postpaid Inflow M1 B2C : DTAC - GEO Channel,DTAC,...,20260202,20260202,1118.0,8676.857333,260305.72,30,N,"PTT : Thanyaburi, Nong Suea, Lam Luk Ka",GX1,BMA : West
1,2025,20254,202511,2025045,20251104,POSTPAID,01035652,DB2R010500CG,Postpaid Inflow M1 B2C : DTAC - GEO Channel,DTAC,...,20260202,20260202,4493.0,8676.857333,260305.72,30,N,"PTT : Thanyaburi, Nong Suea, Lam Luk Ka",GX1,BMA : West
2,2025,20254,202511,2025044,20251101,POSTPAID,01035652,DB2R010500CG,Postpaid Inflow M1 B2C : DTAC - GEO Channel,DTAC,...,20260202,20260202,5643.0,8676.857333,260305.72,30,BM,"PTT : Thanyaburi, Nong Suea, Lam Luk Ka",GX1,BMA : West
3,2025,20254,202511,2025045,20251106,POSTPAID,01035652,DB2R010500CG,Postpaid Inflow M1 B2C : DTAC - GEO Channel,DTAC,...,20260202,20260202,4892.0,8676.857333,260305.72,30,N,"PTT : Thanyaburi, Nong Suea, Lam Luk Ka",GX1,BMA : West
4,2025,20254,202511,2025045,20251103,POSTPAID,01035652,DB2R010500CG,Postpaid Inflow M1 B2C : DTAC - GEO Channel,DTAC,...,20260202,20260202,4094.0,8676.857333,260305.72,30,N,"PTT : Thanyaburi, Nong Suea, Lam Luk Ka",GX1,BMA : West
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948889,2025,20251,202502,2025007,20250213,PREPAID,01032872,DB1S000106CS,Prepaid No of AP In Month : DTAC,DTAC,...,20251023,20251023,10662.0,NaN,NaN,28,N,East,None,None
1948890,2025,20251,202502,2025008,20250220,PREPAID,01032872,DB1S000106CS,Prepaid No of AP In Month : DTAC,DTAC,...,20251023,20251023,9740.0,NaN,NaN,28,N,East,None,None
1948891,2025,20251,202502,2025008,20250222,PREPAID,01032872,DB1S000106CS,Prepaid No of AP In Month : DTAC,DTAC,...,20251023,20251023,8960.0,NaN,NaN,28,N,East,None,None
1948892,2025,20251,202502,2025009,20250224,PREPAID,01032872,DB1S000106CS,Prepaid No of AP In Month : DTAC,DTAC,...,20251023,20251023,7704.0,NaN,NaN,28,N,East,None,None
